In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import various libraries and modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import re
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
nltk.download('stopwords')
from sklearn.model_selection import train_test_split


Train and test data is read using pandas

In [ ]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
test.head()

In [ ]:
train

# Data Cleaning

we could see from above data frame is that the tweet data is not much cleaned.
What is meaning of cleaned data then?
for machine to convert each word as a vector it has to know the meaning of that word or we can say it will present is dictonary of library,So we will only takes those word which machines knows, not some random slangs.

here we will use regular expression to remove special symbols, also use NLTK stemming,lowercase words etc etc we will clean the tweet texts.

In [ ]:
#text_cleaning(df):
    

This is the function I used to cleaned the data

In [ ]:
def cleaningfun(df):
    corpus=[]
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    for sent in list(df['text']):
    
        sent=re.sub("\w*\@\w*", "", sent)
        sent=emoji_pattern.sub(r'', sent)
        sent=re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ",sent)
    
        sent=re.sub(r'<.*?>','',sent)
        sent=re.sub("[^a-zA-Z]+"," ",sent)
        sent=sent.lower()
        sent=sent.split()
        sent=[ ps.stem(word) for word in sent if not word in stopwords.words('english')]
        sent = ' '.join(sent)
 
    
        corpus.append(sent)
    return corpus
    

Applied on Test and Train data

In [ ]:
corpus_train=cleaningfun(train)
corpus_test=cleaningfun(test)

In [ ]:
hidden_train=pd.DataFrame(corpus_train,index=train.id,columns=['text'])
hidden_test=pd.DataFrame(corpus_test,index=test.id,columns=['text'])
hidden_train

We cannot perform classification operation on directly on text data because all the operation use mathamatical concepts for which we required numbers or vectors to be precise. We use TFIDF vectoriser here imported from SKlearn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_train = TfidfVectorizer(max_features = 2500)
Mat_train= vectorizer_train.fit_transform(corpus_train)
#len(vectorizer.get_feature_names())
tfidf_tokens_train = vectorizer_train.get_feature_names()
Mat_train.shape

7613 rows and 2500 columns, these 7613 rows are the text sentences that we have given and the label of each columns is the word

The vectoriser is fit with only Train dataset and we will transform both training and testing data with it, do not perform fit operation on testing data, if performed fit on test data we will get different no. of columns from training data and then the classifier will give error.

In [ ]:

Mat_test= vectorizer_train.transform(corpus_test)
#len(vectorizer.get_feature_names())
#tfidf_tokens_test = vectorizer_test.get_feature_names()
Mat_test.shape

lets define the X and target y

In [ ]:
X = pd.DataFrame(data = Mat_train.toarray(),index =train.id,columns = tfidf_tokens_train)
X

In [ ]:
test_data=pd.DataFrame(data = Mat_test.toarray(),index =test.id,columns = tfidf_tokens_train)
test_data.head()

In [ ]:
y=train['target']

We will do train validation data split 

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=20)
X_train.info()

I have used here Naive bais as Classification algo. here and accuracy of model is predicted using validation data

In [ ]:
# training the model on training set
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
  
# making predictions on the testing set
y_pred = gnb.predict(X_val)
  
# comparing actual response values (y_test) with predicted response values (y_pred)
from sklearn import metrics
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_val, y_pred)*100)

Trained model used to predict test data targets also and save them

In [ ]:
y_test = gnb.predict(test_data)
y_test

In [ ]:
pd.DataFrame({
    'id':test.id,
    'target':y_test
}).to_csv('submission.csv',index=False)